In [1]:
import tensorflow.compat.v1 as tf
import lucid_kietzmannlab.modelzoo.vision_models as models
from ipywidgets import interact, Dropdown, IntSlider
from lucid_kietzmannlab.utils import interactive_visualization, batch_visualization, plot_images
tf.compat.v1.disable_eager_execution()


In [2]:
model_checkpoint_dir="/Users/vkapoor/Downloads/models/AlexNet/training_seed_05"
training_seed = 5
model = models.AlexNetv2(model_checkpoint_dir=model_checkpoint_dir, training_seed = training_seed)


Loading model with name /Users/vkapoor/Downloads/models/AlexNet/training_seed_05/model.ckpt_epoch89
INFO:tensorflow:Restoring parameters from /Users/vkapoor/Downloads/models/AlexNet/training_seed_05/model.ckpt_epoch89


2024-06-12 20:17:51.701389: W tensorflow/core/common_runtime/graph_constructor.cc:1595] Importing a graph with a lower producer version 38 into an existing graph with producer version 1766. Shape inference will have run different parts of the graph with different producer versions.
2024-06-12 20:17:53.863898: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-06-12 20:17:55.776369: W tensorflow/c/c_api.cc:305] Operation '{name:'alexnet_v2/fc8/biases/Momentum/Assign' id:1952 op device:{requested: '', assigned: ''} def:{{{node alexnet_v2/fc8/biases/Momentum/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@alexnet_v2/fc8/biases"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](alexnet_v2/fc8/biases/Momentum, alexnet_v2/fc8/biases/Momentum/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either 

In [3]:
layer_shape_dict = model.layer_shape_dict
def visualize(layer_name, channel):
    interactive_visualization(model,layer_name, channel, scope = '', channels_first = True)   

def visualize_all():
    layer_name = current_dropdown_value({'new': layer_dropdown.value})
    image_channel = batch_visualization(model, layer_name, channel_slider, scope = '', channels_first = True)
    return image_channel              
                              

        
# Create dropdown menu for layer selection
layer_dropdown = Dropdown(options=list(layer_shape_dict.keys()), description='Layer:')

# Create slider for channel selection
channel_slider = IntSlider(min=0, max=0, description='Channel:')
        
        
def update_channel_slider(change):
    layer_name = change.new
    if layer_name in layer_shape_dict:
        
        max_channel = layer_shape_dict[layer_name][-1] - 1
        channel_slider.max = max_channel
        
        
def current_slider_value(*args):
    return channel_slider.value


def current_dropdown_value(change):
    return change['new']


channel_slider.observe(current_slider_value, names='value')
layer_dropdown.observe(current_dropdown_value, names='value')
        

       

In [4]:
layer_dropdown.observe(update_channel_slider, names='value')

# Create an interactive visualization
interact(visualize, layer_name=layer_dropdown, channel=channel_slider)     

interactive(children=(Dropdown(description='Layer:', options=('tower_0/alexnet_v2/conv1/Conv2D', 'tower_0/alex…

<function __main__.visualize(layer_name, channel)>

In [ ]:
image_channel = visualize_all()

In [ ]:

if image_channel:
    plot_images(image_channel)   

In [ ]:
layer_shape_dict.items()

In [4]:
visualize('alexnet_v2/conv1/weights',0)

2024-06-12 20:18:11.796131: W tensorflow/core/common_runtime/graph_constructor.cc:1595] Importing a graph with a lower producer version 38 into an existing graph with producer version 1766. Shape inference will have run different parts of the graph with different producer versions.


ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'Variable:0' shape=(2, 128, 3, 65) dtype=float32>"] and loss Tensor("Neg:0", shape=(), dtype=float32).

In [18]:
layer_name = 'alexnet_v2/conv1/weights'
graph = tf.compat.v1.get_default_graph()
tensor = graph.get_tensor_by_name(f"{layer_name}:0")
tensor_shape = tensor.shape
print(tensor_shape, tensor_shape[-1])

(11, 11, 3, 64) 64


In [ ]:
graph = tf.compat.v1.get_default_graph()
for op in graph.operations:
    print(op.name)

In [5]:

graph = tf.get_default_graph()
inputs_placeholder = graph.get_tensor_by_name('Placeholder:0')



In [6]:
inputs_placeholder.shape

TensorShape([None, 3, 227, 227])

In [7]:
vars = tf.trainable_variables()

In [8]:
vars

[<tf.Variable 'alexnet_v2/conv1/weights:0' shape=(11, 11, 3, 64) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/conv1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/conv2/weights:0' shape=(5, 5, 64, 192) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/conv2/biases:0' shape=(192,) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/conv3/weights:0' shape=(3, 3, 192, 384) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/conv3/biases:0' shape=(384,) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/conv4/weights:0' shape=(3, 3, 384, 384) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/conv4/biases:0' shape=(384,) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/conv5/weights:0' shape=(3, 3, 384, 256) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/conv5/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/fc6/weights:0' shape=(5, 5, 256, 4096) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/fc6/biases:0' shape=(4096,) dtype=float32_ref>,
 <tf.Variable 'alexnet_v2/fc7/weights:0

In [15]:
test_placeholder = graph.get_tensor_by_name('alexnet_v2/conv1/weights:0')

In [10]:
test_placeholder.shape

TensorShape([11, 11, 3, 64])

In [ ]:
for op in graph.get_operations():
        try:
          if graph.get_tensor_by_name(f'{op.name}:0').shape[1] == 565: 
             print(op.name, graph.get_tensor_by_name(f'{op.name}:0').shape)
        except:
            pass     

In [ ]:
with tf.Graph().as_default() as graph:
    tf.import_graph_def(model.graph_def, name="")
    writer = tf.summary.FileWriter('AlexNetv2', graph)
    writer.close()